In [2]:
import numpy as np
import pandas as pd
import urllib.parse as urlparse
import requests
import json

In [4]:
def make_request_url(service_key, url_param):
    url = "http://openapi.tago.go.kr/openapi/service/BusRouteInfoInqireService/getRouteAcctoThrghSttnList"
    params = urlparse.urlencode(url_param)
    url += '?' +"serviceKey=" + service_key + "&" + params
    return url

In [5]:
# param
cityCode = 34010
numOfRows = 50
pageNo = 1
_type = 'json'

# service key
key = 'YxN%2FPifj5Yhgp6Qb6xLFamJJFFZF1UThR2SlXB6eKCTEgpv6PVN7oktz%2Bzt6JTxNuPbB3pIjPNHwAp%2BBl1RA0Q%3D%3D'

In [6]:
route_id_df = pd.read_csv('../csv/route_id.csv')
route_id_df

,Unnamed: 0,routeno,routeid
0,0,11,CAB285000006
1,1,11,CAB285000007
2,2,12,CAB285000008
3,3,12,CAB285000009
4,4,13,CAB285000010
5,5,13,CAB285000011
6,6,14,CAB285000012
7,7,14,CAB285000013
8,8,24,CAB285000024
9,9,24,CAB285000025


In [7]:
route_ids = route_id_df['routeid']
route_ids

0     CAB285000006
1     CAB285000007
2     CAB285000008
3     CAB285000009
4     CAB285000010
5     CAB285000011
6     CAB285000012
7     CAB285000013
8     CAB285000024
9     CAB285000025
10    CAB285000293
11    CAB285000294
Name: routeid, dtype: object

In [8]:
node_ids = dict() # {'route_id': [# 해당 노선의 경유 정류소 ids]}

for routeId in route_ids:
    param = {'cityCode': cityCode, 'routeId': routeId, 'numOfRows': numOfRows, 'pageNo': pageNo, '_type': _type}
    
    # make request url & get response
    request_url = make_request_url(key, param)
    response = requests.get(url=request_url)

    # parse json
    r_dict = json.loads(response.text)
    r_response = r_dict.get("response")
    r_body = r_response.get("body")
    r_items = r_body.get("items")
    r_item = r_items.get("item")
    
    # append node_ids to list
    node_ids[routeId] = list()
    for item in r_item:
        node_ids[routeId].append(item['nodeid'])

In [10]:
node_ids

{'CAB285000006': ['CAB285000930',
  'CAB285000928',
  'CAB285000926',
  'CAB285000940',
  'CAB285000917',
  'CAB285000920',
  'CAB285000921',
  'CAB285000923',
  'CAB285000879',
  'CAB285000908',
  'CAB285000905',
  'CAB285000903',
  'CAB285001519',
  'CAB285000901',
  'CAB285000897',
  'CAB285000899',
  'CAB285000657',
  'CAB285001765',
  'CAB285002264',
  'CAB285000676',
  'CAB285000663',
  'CAB285000666',
  'CAB285000668',
  'CAB285000678',
  'CAB285000654',
  'CAB285000651',
  'CAB285000650',
  'CAB285010014',
  'CAB285000706',
  'CAB285001061',
  'CAB285001477',
  'CAB285001053',
  'CAB285001478',
  'CAB285071397',
  'CAB285001056',
  'CAB285001058',
  'CAB285001060',
  'CAB285000982',
  'CAB285010023',
  'CAB285001483',
  'CAB285001034',
  'CAB285001030',
  'CAB285001032'],
 'CAB285000007': ['CAB285001033',
  'CAB285001031',
  'CAB285001035',
  'CAB285000979',
  'CAB285000980',
  'CAB285000981',
  'CAB285001059',
  'CAB285001057',
  'CAB285001055',
  'CAB285001478',
  'CAB2850713

### convert node_ids(dict) to dataframe

In [36]:
node_ids_df = pd.DataFrame(columns=['routeid', 'node_ids'])
# pd.DataFrame(columns)
node_ids_df

,routeid,node_ids


In [37]:
node_ids_df['routeid'] = node_ids.keys()
node_ids_df

,routeid,node_ids
0,CAB285000006,NaN
1,CAB285000007,NaN
2,CAB285000008,NaN
3,CAB285000009,NaN
4,CAB285000010,NaN
5,CAB285000011,NaN
6,CAB285000012,NaN
7,CAB285000013,NaN
8,CAB285000024,NaN
9,CAB285000025,NaN


In [38]:
for i, n_ids in enumerate(node_ids.values()):
    node_ids_df.at[i, 'node_ids'] = n_ids
    
node_ids_df

,routeid,node_ids
0,CAB285000006,"[CAB285000930, CAB285000928, CAB285000926, CAB..."
1,CAB285000007,"[CAB285001033, CAB285001031, CAB285001035, CAB..."
2,CAB285000008,"[CAB285010020, CAB285000873, CAB285000872, CAB..."
3,CAB285000009,"[CAB285000784, CAB285000780, CAB285001969, CAB..."
4,CAB285000010,"[CAB285000640, CAB285000641, CAB285000642, CAB..."
5,CAB285000011,"[CAB288002022, CAB288002048, CAB288002219, CAB..."
6,CAB285000012,"[CAB285010020, CAB285000873, CAB285000872, CAB..."
7,CAB285000013,"[CAB285001598, CAB285001597, CAB285002282, CAB..."
8,CAB285000024,"[CAB285001045, CAB285001044, CAB285002190, CAB..."
9,CAB285000025,"[CAB285010146, CAB285000318, CAB285000317, CAB..."


In [39]:
node_ids_df.to_csv('../csv/all_node_ids.csv')